<a href="https://colab.research.google.com/github/tytybella/Amazon_review_nlp/blob/main/topic_modeling_LDA_%26_BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pyLDAvis

In [ ]:
!pip install numpy --upgrade

In [ ]:
!pip install gensim

In [ ]:
!pip install --upgrade pandas gensim pyldavis

In [1]:
import gensim
from gensim import corpora
from gensim.models import LdaModel
import pandas as pd

# 0. Data prepartion

In [ ]:
from google.colab import files

# Upload files
files.upload()  # need cleaned_data.csv

In [60]:
data = pd.read_csv('cleaned_data.csv')
#only keep the 'overall' column and 'basic_cleaned_reviewText'
df = data[['category', 'cleaned_reviewText', 'overall']]
df.sample(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,category,cleaned_reviewText,overall
16830,"Clothing,ShoesJewelry",color nice top supportive however little bow b...,2.0
35922,"Clothing,ShoesJewelry",buy lot type sunglass fit glass le intrusive m...,5.0
43325,"Clothing,ShoesJewelry",worked great daughter,5.0
54657,"Clothing,ShoesJewelry",real big dwight chute syndrome suffer smaller ...,2.0
49429,"Clothing,ShoesJewelry",cute well made friend really liked !,4.0


In [7]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [126]:
df.info()
df = df.dropna(subset=['cleaned_reviewText', 'category'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57397 entries, 0 to 57396
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   category            55306 non-null  object 
 1   cleaned_reviewText  57283 non-null  object 
 2   overall             57397 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 55195 entries, 0 to 57396
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   category            55195 non-null  object 
 1   cleaned_reviewText  55195 non-null  object 
 2   overall             55195 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.7+ MB


In [127]:
df['tokenized_text'] = df['cleaned_reviewText'].apply(lambda x: word_tokenize(x))
df.sample(5)

,category,cleaned_reviewText,overall,tokenized_text
16068,"Clothing,ShoesJewelry",far good ! really like comfort especially supp...,5.0,"[far, good, !, really, like, comfort, especial..."
25744,"Clothing,ShoesJewelry",wife worn prescription glass since middle scho...,5.0,"[wife, worn, prescription, glass, since, middl..."
33316,"Clothing,ShoesJewelry",overall really nice bag one thing could improv...,4.0,"[overall, really, nice, bag, one, thing, could..."
3649,"Clothing,ShoesJewelry",god,5.0,[god]
28448,"Clothing,ShoesJewelry",love product mess los clothes,5.0,"[love, product, mess, los, clothes]"


# 1. LDA Topic Modeling:

vectorizer used gensim's copora.dictionary and Gensim's doc2bow, which is a  Bag-of-Words (BoW) method

## 1.1 LDA topic modeling for the  whole review datasets

### build model

In [47]:
# Create Dictionary
id2word = corpora.Dictionary(df['tokenized_text'])
# Create Corpus
texts = df['tokenized_text']
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
# Human readable format of corpus (term-frequency)
# There is nothing magical about 2500:2510, I just wanted to examine a random location
print ([[(id2word[id], freq) for id, freq in cp] for cp in corpus[2500:2512]])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[('well', 1), ('comfortable', 1), ('fit', 1)], [('love', 1), ('realize', 1), ('got', 1), ('pocket', 1), ('sweat', 1), ('dis', 1)], [('like', 1), ('one', 1), ('size', 2), ('large', 1), ('however', 1), ('im', 1), ('look', 1), ('short', 1), ('would', 1), ('quality', 1), ('really', 1), ('waist', 1), ('got', 1), ('get', 1), ('made', 1), ('al', 2), ('sizing', 1), ('smaller', 1), ('long', 1), ('leg', 2), ('room', 1), ('actually', 1), ('amount', 1), ('elastic', 1), ('extra', 1), ('hold', 1), ('based', 1), ('pair', 1), ('bigger', 1), ('listed', 1), ('experience', 1), ('nicely', 1), ('past', 1), ('soon', 1), ('accommodate', 1), ('sweat', 1), ('thankfully', 1), ('eg', 1), ('gigantic', 1), ('hulk', 1), ('roll', 1)], [('fit', 1), ('nice', 1), ('easy', 1), ('expected', 1), ('color', 1), ('lighter', 1), ('due', 1), ('possibly', 1), ('stain', 1)], [('baggy', 1), ('pant', 1)], [('fit', 1), ('great', 1)], [('fit', 1), ('fabric', 1), ('quality', 1), ('nothing', 1), ('fleece', 1), ('want', 1), ('paid', 1

In [48]:
reviews_corpus = dict(id2word)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tried num_topics=10: can't visulize, some topic empty

tried num_topics=5: three topics overlap together

choosed num_topic=3

In [29]:
# takes 16 mins to run


# Build LDA model
def build_lda_model(corpus, id2word, num_topics = 3):
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=num_topics,
                                            random_state=42,
                                            update_every=1,
                                            chunksize=5,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
  return lda_model

lda_model = build_lda_model(corpus, id2word)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:

lda_model.print_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.025*"good" + 0.018*"time" + 0.017*"get" + 0.013*"product" + 0.012*"even" + 0.011*"could" + 0.010*"much" + 0.010*"see" + 0.009*"bag" + 0.009*"back"'),
 (1,
  '0.103*"!" + 0.042*"like" + 0.038*"size" + 0.029*"wear" + 0.020*"one" + 0.020*"well" + 0.019*"little" + 0.018*"bought" + 0.017*"would" + 0.016*"ordered"'),
 (2,
  '0.053*"fit" + 0.050*"love" + 0.037*"great" + 0.025*"look" + 0.024*"small" + 0.024*"nice" + 0.022*"quality" + 0.019*"cute" + 0.019*"color" + 0.018*"perfect"')]

In [31]:
doc_lda = lda_model[corpus]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Visualize result

In [ ]:
!pip install pyLDAvis==3.2.1

In [27]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, n_jobs=1)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)



Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

(A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart. )

## 1.2 LDA Topic modeling for each category

### Build model and visualization

In [ ]:
import warnings

# Ignore specific DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module='ipykernel')

In [124]:
def get_corpus_id2word(text_data):
  # Create Dictionary
  cur_id2word = corpora.Dictionary(text_data['tokenized_text'])
  # Create Corpus
  texts = text_data['tokenized_text']
  # Term Document Frequency
  cur_corpus = [cur_id2word.doc2bow(text) for text in texts]
  return cur_corpus, cur_id2word

def get_data(category, rating_filter=False):
  '''get only this category's data'''
  # filter category
  filtered_df = df[df['category'] == category]
  # filter only rating 4 and 5
  if rating_filter:
    filtered_df = filtered_df[filtered_df['overall'].isin([4, 5])]
  return filtered_df



In [122]:
print(df.category.unique())


['Clothing,ShoesJewelry' nan 'Clothing,Shoesamp;Jewelry' 'SportsOutdoors'
 'HealthHousehold' ' Tools & Home Improvement ' 'Books' 'AmazonLaunchpad'
 'CellPhonesAccessories' 'BeautyPersonalCare' ' Home & Kitchen '
 ' Arts, Crafts & Sewing ' ' Toys & Games ' ' Office Products '
 'IndustrialScientific' ' Kitchen & Dining ' 'Electronics' 'Baby'
 ' Automotive ' 'Arts,CraftsSewing' 'HomeKitchen']


In [128]:
category = 'SportsOutdoors'

cur_corpus, cur_id2word = get_corpus_id2word(get_data(category))
cur_lda_model = build_lda_model(cur_corpus, cur_id2word, num_topics = 3)

In [129]:
cur_lda_model.print_topics()

[(0,
  '0.056*"great" + 0.028*"quality" + 0.021*"product" + 0.019*"make" + 0.018*"price" + 0.015*"sock" + 0.014*"enough" + 0.013*"purse" + 0.012*"thats" + 0.009*"work"'),
 (1,
  '0.059*"!" + 0.032*"like" + 0.031*"fit" + 0.018*"would" + 0.017*"love" + 0.016*"good" + 0.015*"one" + 0.014*"well" + 0.012*"bag" + 0.011*"little"'),
 (2,
  '0.028*"lens" + 0.022*"look" + 0.020*"nice" + 0.015*"much" + 0.013*"something" + 0.011*"buy" + 0.011*"get" + 0.011*"original" + 0.011*"got" + 0.011*"made"')]

In [130]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(cur_lda_model, cur_corpus, cur_id2word, n_jobs=1)
pyLDAvis.display(vis)



/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


### Further explore this catrogy and the category's positive reviews topics

In [131]:
category = 'SportsOutdoors'

cur_corpus, cur_id2word = get_corpus_id2word(get_data(category, rating_filter=True))
cur_lda_model = build_lda_model(cur_corpus, cur_id2word, num_topics = 3)

In [132]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(cur_lda_model, cur_corpus, cur_id2word, n_jobs=1)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


## 1.3 LDA Topic modeling for each brand

In [134]:
df2 = data[['brand', 'cleaned_reviewText', 'overall']]
df2.sample(5)

,brand,cleaned_reviewText,overall
12158,MJ Metals Jewelry,like ring use size instead,4.0
50898,ALYGNE,cute,5.0
18502,LeatherBoss,buying every six month year must good,5.0
55838,Generic,beware completely see,1.0
13764,TOP HEADWEAR,looked little big infant cap line toddler cap,4.0


In [136]:
df2 = df2.dropna(subset=['cleaned_reviewText', 'brand', 'overall'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41125 entries, 11 to 57396
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   brand               41125 non-null  object 
 1   cleaned_reviewText  41125 non-null  object 
 2   overall             41125 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.3+ MB


In [137]:
len(df2.brand.unique())

4723

In [138]:
df2.brand.sample(1)

8893    Plum Feathers
Name: brand, dtype: object

In [139]:
df2['tokenized_text'] = df2['cleaned_reviewText'].apply(lambda x: word_tokenize(x))
df2.sample(5)

,brand,cleaned_reviewText,overall,tokenized_text
38678,The North Face,fit like glove highly recommend nothing compar...,5.0,"[fit, like, glove, highly, recommend, nothing,..."
25129,Working Class,mind coat large child grow fine love coat plea...,5.0,"[mind, coat, large, child, grow, fine, love, c..."
34979,Flora,received petticoat today got ta say im pretty ...,1.0,"[received, petticoat, today, got, ta, say, im,..."
20224,Skylarking,give five star came fast shared bunch friend f...,5.0,"[give, five, star, came, fast, shared, bunch, ..."
34734,Fanmis,wow watch look like something world something ...,5.0,"[wow, watch, look, like, something, world, som..."


In [140]:
def get_data2(brand, rating_filter=False):
  '''get only this category's data'''
  # filter category
  filtered_df = df2[df2['brand'] == brand]
  # filter only rating 4 and 5
  if rating_filter:
    filtered_df = filtered_df[filtered_df['overall'].isin([1])]
  return filtered_df

In [141]:
def get_corpus_id2word(text_data):
  # Create Dictionary
  cur_id2word = corpora.Dictionary(text_data['tokenized_text'])
  # Create Corpus
  texts = text_data['tokenized_text']
  # Term Document Frequency
  cur_corpus = [cur_id2word.doc2bow(text) for text in texts]
  return cur_corpus, cur_id2word

In [142]:
df2.brand.sample(1)

46646    Jigg And Roll
Name: brand, dtype: object

In [143]:
brand_counts = df2.groupby('brand').size().sort_values(ascending=False).head(10)
print(brand_counts)

brand
i play.                       878
Pierced Owl                   631
Angerella                     630
MJ Metals Jewelry             439
LaSuiveur                     356
JASGOOD                       300
Fawziya                       286
Groove Plus Life              279
WearMe Pro                    276
Mystic Metals Body Jewelry    271
dtype: int64


## choose a brand here, and see topics

In [144]:
brand = 'i play.'

cur_corpus2, cur_id2word2 = get_corpus_id2word(get_data2(brand))
cur_lda_model = build_lda_model(cur_corpus2, cur_id2word2, num_topics = 3)

In [145]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(cur_lda_model, cur_corpus2, cur_id2word2, n_jobs=1)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


## further to explore this brand's negative reviews topics

In [146]:
cur_corpus2, cur_id2word2 = get_corpus_id2word(get_data2(brand, rating_filter=True))
cur_lda_model = build_lda_model(cur_corpus2, cur_id2word2, num_topics = 3)

In [147]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(cur_lda_model, cur_corpus2, cur_id2word2, n_jobs=1)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


# 2. BerTopic

no need to do vectorizer, BERTopic uses transformer-based models to create dense vector representations of text documents.

(Limitation: It would be better to use basic_cleaned_data on raw data or basic_cleaned_data, but here used deep cleaned data for convenience)

## 2.1 BerTopic modeling for the whole review dataset

In [ ]:
!pip install bertopic

In [149]:
# Assuming 'df' is your DataFrame and 'text_column' is the name of the column containing text
documents = df['cleaned_reviewText'].tolist()

In [221]:
from bertopic import BERTopic

# Create a BERTopic model
topic_model = BERTopic(min_topic_size=100)   #  when min_topic_size=10, has 715 topics, too much, increased to 100;
                                              # only a 30000+ size huge topic and some other tiny topics.
# Fit the model
topics, probabilities = topic_model.fit_transform(documents)

#### Result has 25 topics, need to further reduce topics

In [170]:
# help(topic_model.reduce_topics)

In [222]:
# Get the topics
topic_info = topic_model.get_topic_info()
# Print the topics
print(topic_info)

    Topic  Count                               Name  \
0      -1  18492             -1_fit_like_cute_great   
1       0   3143          0_gift_daughter_old_loved   
2       1   2258           1_shoe_foot_boot_insoles   
3       2   1713         2_shirt_collar_sweater_fit   
4       3   1621       3_size_small_ordered_smaller   
..    ...    ...                                ...   
80     79    114        79_mask_face_masquerade_eye   
81     80    109            80_star_give_zero_could   
82     81    106       81_pin_lapel_insanity_brooch   
83     82    102     82_cheap_cheaply_made_southern   
84     83    100  83_checkbook_check_cover_register   

                                       Representation  \
0   [fit, like, cute, great, quality, good, look, ...   
1   [gift, daughter, old, loved, son, year, month,...   
2   [shoe, foot, boot, insoles, pain, heel, toe, a...   
3   [shirt, collar, sweater, fit, nice, tee, size,...   
4   [size, small, ordered, smaller, al, large, siz... 

In [225]:
# reduce topics
topic_model.reduce_topics(docs=documents, nr_topics=10)

# Access the updated topics and probabilities
new_topics = topic_model.topics_
new_probs = topic_model.probabilities_

In [226]:
# Visualize topics
topic_model.visualize_topics()

## 2.2 BerTopic modeling for each category

In [175]:
print(df.category.unique())

['Clothing,ShoesJewelry' 'Clothing,Shoesamp;Jewelry' 'SportsOutdoors'
 'HealthHousehold' ' Tools & Home Improvement ' 'Books' 'AmazonLaunchpad'
 'CellPhonesAccessories' 'BeautyPersonalCare' ' Home & Kitchen '
 ' Arts, Crafts & Sewing ' ' Toys & Games ' ' Office Products '
 'IndustrialScientific' ' Kitchen & Dining ' 'Electronics' 'Baby'
 ' Automotive ' 'Arts,CraftsSewing' 'HomeKitchen']


In [177]:
category = 'SportsOutdoors'

# Filter the DataFrame
filtered_df = df[df['category'] == category]
documents2 = filtered_df['cleaned_reviewText'].tolist()

In [185]:
# Create a BERTopic model
topic_model2 = BERTopic(min_topic_size=10)   #  when min_topic_size=10, has 715 topics, too much, increased to 300
# Fit the model
topics2, probabilities2 = topic_model2.fit_transform(documents2)

In [186]:
# Get the topics
topic_info2 = topic_model2.get_topic_info()
# Print the topics
print(topic_info2)

    Topic  Count                               Name  \
0      -1     95          -1_color_love_like_really   
1       0    103       0_fit_short_size_comfortable   
2       1     46            1_lens_glass_frame_pair   
3       2     39        2_ring_beautiful_gift_stone   
4       3     36             3_shoe_sock_pedal_foot   
5       4     30        4_nice_beautiful_cute_super   
6       5     22             5_hat_quality_fit_head   
7       6     22           6_love_wife_loved_loveit   
8       7     21           7_bag_purse_zipper_carry   
9       8     18      8_product_pleased_great_sweet   
10      9     15          9_watch_strap_band_signal   
11     10     14  10_expected_described_work_bigger   
12     11     13      11_quality_good_price_product   

                                       Representation  \
0   [color, love, like, really, cute, look, even, ...   
1   [fit, short, size, comfortable, great, ordered...   
2   [lens, glass, frame, pair, valley, original, e...   
3

In [203]:
# reduce topics
topic_model2.reduce_topics(docs=documents2, nr_topics=5)

# Access the updated topics and probabilities
new_topics2 = topic_model2.topics_
new_probs2 = topic_model2.probabilities_

In [204]:
# Visualize topics
topic_model2.visualize_topics()

## 2.3 Bertopic modeling for each brand

In [191]:
print(df2.brand.unique())

['Aeromax' 'California Costumes' 'Plush Necessities' ... 'Anna-Kaci'
 'Ivysport' 'Edith Windsor']


In [192]:
brand_counts

brand
i play.                       878
Pierced Owl                   631
Angerella                     630
MJ Metals Jewelry             439
LaSuiveur                     356
JASGOOD                       300
Fawziya                       286
Groove Plus Life              279
WearMe Pro                    276
Mystic Metals Body Jewelry    271
dtype: int64

In [210]:
brand = 'i play.'

# Filter the DataFrame
filtered_df3 = df2[df2['brand'] == brand]
documents3 = filtered_df['cleaned_reviewText'].tolist()

In [211]:
# Create a BERTopic model
topic_model3 = BERTopic(min_topic_size=10)   #  when min_topic_size=10, has 715 topics, too much, increased to 300
# Fit the model
topics3, probabilities3 = topic_model3.fit_transform(documents3)

In [212]:
# Get the topics
topic_info3 = topic_model3.get_topic_info()
# Print the topics
print(topic_info3)

    Topic  Count                           Name  \
0      -1    110        -1_like_cute_love_color   
1       0     72  0_fit_short_great_comfortable   
2       1     46        1_lens_glass_frame_pair   
3       2     37   2_product_quality_good_great   
4       3     36         3_shoe_sock_pedal_foot   
5       4     31    4_nice_beautiful_cute_super   
6       5     24   5_gift_daughter_cute_costume   
7       6     23       6_love_wife_loved_loveit   
8       7     21       7_hat_quality_fit_helmet   
9       8     21       8_bag_purse_zipper_carry   
10      9     20  9_ring_beautiful_silver_color   
11     10     18        10_small_size_large_run   
12     11     15     11_watch_strap_band_signal   

                                       Representation  \
0   [like, cute, love, color, feel, old, white, ex...   
1   [fit, short, great, comfortable, ordered, perf...   
2   [lens, glass, frame, pair, valley, original, e...   
3   [product, quality, good, great, pleased, exact...   


In [213]:
# reduce topics
topic_model3.reduce_topics(docs=documents3, nr_topics=5)

# Access the updated topics and probabilities
new_topics3 = topic_model3.topics_
new_probs3 = topic_model3.probabilities_

In [214]:
# Visualize topics
topic_model3.visualize_topics()